# E-commerce Sales Dashboard Analysis

**Objective:** Analyze £9.7M in transactions to identify revenue drivers, seasonal patterns, and high-value customer segments.

**Dataset:** UCI Online Retail II (2009-2011)  
**Author:** [Your Name]  
**Date:** [Date]

## 1. Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print('✅ Libraries loaded successfully')

In [ ]:
# Load data
# Option 1: Load from processed data
try:
    df = pd.read_csv('../data/processed/cleaned_transactions.csv')
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    print(f'✅ Loaded processed data: {len(df):,} rows')
except FileNotFoundError:
    # Option 2: Load sample data
    df = pd.read_csv('../data/sample/sample_data.csv')
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    print(f'⚠️ Using sample data: {len(df):,} rows')

df.head()

## 2. Data Overview

In [ ]:
# Basic info
print('📊 Dataset Summary')
print('=' * 50)
print(f"Total Transactions: {len(df):,}")
print(f"Unique Customers: {df['Customer ID'].nunique():,}")
print(f"Unique Products: {df['StockCode'].nunique():,}")
print(f"Countries: {df['Country'].nunique()}")
print(f"Date Range: {df['InvoiceDate'].min().date()} to {df['InvoiceDate'].max().date()}")
print(f"\nTotal Revenue: £{df['Revenue'].sum():,.2f}")

In [ ]:
# Data types and missing values
print('\n📋 Data Types and Missing Values')
print('=' * 50)
info_df = pd.DataFrame({
    'Type': df.dtypes,
    'Non-Null': df.count(),
    'Missing': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df) * 100).round(2)
})
info_df

## 3. Revenue Analysis

In [ ]:
# Monthly revenue trend
monthly_revenue = df.groupby(df['InvoiceDate'].dt.to_period('M')).agg({
    'Revenue': 'sum',
    'Invoice': 'nunique',
    'Customer ID': 'nunique'
}).reset_index()
monthly_revenue.columns = ['Month', 'Revenue', 'Orders', 'Customers']
monthly_revenue['Month'] = monthly_revenue['Month'].astype(str)

# Plot
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=('Monthly Revenue', 'Monthly Orders & Customers'))

fig.add_trace(
    go.Scatter(x=monthly_revenue['Month'], y=monthly_revenue['Revenue'],
               mode='lines+markers', name='Revenue', fill='tozeroy'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=monthly_revenue['Month'], y=monthly_revenue['Orders'], name='Orders'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=monthly_revenue['Month'], y=monthly_revenue['Customers'],
               mode='lines+markers', name='Customers', yaxis='y3'),
    row=2, col=1
)

fig.update_layout(height=600, title_text='Monthly Performance Overview')
fig.show()

In [ ]:
# Key insight: November spike
monthly_revenue['MonthNum'] = pd.to_datetime(monthly_revenue['Month']).dt.month
month_avg = monthly_revenue.groupby('MonthNum')['Revenue'].mean().reset_index()

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_avg['MonthName'] = month_avg['MonthNum'].apply(lambda x: month_names[x-1])

fig = px.bar(month_avg, x='MonthName', y='Revenue',
             title='Average Revenue by Month (Seasonality)',
             labels={'Revenue': 'Average Revenue (£)', 'MonthName': 'Month'})

# Highlight November
colors = ['lightblue'] * 12
colors[10] = 'darkblue'  # November
fig.update_traces(marker_color=colors)

fig.add_annotation(x='Nov', y=month_avg[month_avg['MonthName']=='Nov']['Revenue'].values[0],
                   text='Holiday Shopping Peak', showarrow=True, arrowhead=1)

fig.show()

# Calculate spike
nov_avg = month_avg[month_avg['MonthName']=='Nov']['Revenue'].values[0]
other_avg = month_avg[month_avg['MonthName']!='Nov']['Revenue'].mean()
print(f"\n💡 INSIGHT: November generates {nov_avg/other_avg:.1f}x the average monthly revenue")

## 4. Customer Segmentation (RFM Analysis)

In [ ]:
# Calculate RFM metrics
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'Invoice': 'nunique',  # Frequency
    'Revenue': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

print('📊 RFM Distribution')
rfm.describe()

In [ ]:
# Score each metric (1-5)
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=5, labels=[5, 4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])

# Combined score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Segment labels
def segment_customer(row):
    r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Loyal Customers'
    elif r >= 4 and f <= 2:
        return 'New Customers'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    elif r <= 2 and f <= 2:
        return 'Lost'
    else:
        return 'Potential Loyalists'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)

# Summary
segment_summary = rfm.groupby('Segment').agg({
    'CustomerID': 'count',
    'Monetary': ['sum', 'mean']
}).round(2)
segment_summary.columns = ['Customers', 'Total Revenue', 'Avg Revenue']
segment_summary = segment_summary.sort_values('Total Revenue', ascending=False)

print('👥 Customer Segments')
segment_summary

In [ ]:
# Visualize segments
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'pie'}, {'type':'pie'}]],
                    subplot_titles=['Customers by Segment', 'Revenue by Segment'])

fig.add_trace(
    go.Pie(labels=segment_summary.index, values=segment_summary['Customers'],
           name='Customers'),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=segment_summary.index, values=segment_summary['Total Revenue'],
           name='Revenue'),
    row=1, col=2
)

fig.update_layout(title_text='Customer Segmentation Analysis', height=400)
fig.show()

# 80/20 analysis
top_20_pct_customers = int(len(rfm) * 0.2)
top_customers = rfm.nlargest(top_20_pct_customers, 'Monetary')
top_20_revenue = top_customers['Monetary'].sum()
total_revenue = rfm['Monetary'].sum()

print(f"\n💡 INSIGHT: Top 20% of customers generate {top_20_revenue/total_revenue*100:.0f}% of revenue")

## 5. Geographic Analysis

In [ ]:
# Country breakdown
country_stats = df.groupby('Country').agg({
    'Revenue': 'sum',
    'Customer ID': 'nunique',
    'Invoice': 'nunique'
}).reset_index()
country_stats.columns = ['Country', 'Revenue', 'Customers', 'Orders']
country_stats = country_stats.sort_values('Revenue', ascending=False)
country_stats['Market Share %'] = (country_stats['Revenue'] / country_stats['Revenue'].sum() * 100).round(2)

print('🌍 Top 10 Countries by Revenue')
country_stats.head(10)

In [ ]:
# Visualize (excluding UK for better view of international)
intl_countries = country_stats[country_stats['Country'] != 'United Kingdom'].head(10)

fig = px.bar(intl_countries, x='Revenue', y='Country', orientation='h',
             title='Top 10 International Markets by Revenue',
             labels={'Revenue': 'Revenue (£)'})
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

# UK dominance
uk_revenue = country_stats[country_stats['Country'] == 'United Kingdom']['Revenue'].values[0]
uk_share = country_stats[country_stats['Country'] == 'United Kingdom']['Market Share %'].values[0]
print(f"\n💡 INSIGHT: UK accounts for {uk_share:.0f}% of total revenue (£{uk_revenue:,.0f})")

## 6. Product Analysis

In [ ]:
# Top products
product_stats = df.groupby(['StockCode', 'Description']).agg({
    'Revenue': 'sum',
    'Quantity': 'sum',
    'Invoice': 'nunique'
}).reset_index()
product_stats.columns = ['StockCode', 'Description', 'Revenue', 'Quantity', 'Orders']
product_stats = product_stats.sort_values('Revenue', ascending=False)

print('🏆 Top 10 Products by Revenue')
product_stats.head(10)

In [ ]:
# Visualize top products
top_10 = product_stats.head(10)

fig = px.bar(top_10, x='Revenue', y='Description', orientation='h',
             title='Top 10 Products by Revenue',
             labels={'Revenue': 'Revenue (£)'},
             color='Revenue', color_continuous_scale='Blues')
fig.update_layout(yaxis={'categoryorder': 'total ascending'}, showlegend=False,
                  coloraxis_showscale=False)
fig.show()

## 7. Time Patterns

In [ ]:
# Day of week analysis
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_stats = df.groupby('DayOfWeek')['Revenue'].sum().reset_index()
dow_stats['DayName'] = dow_stats['DayOfWeek'].apply(lambda x: day_names[x])

fig = px.bar(dow_stats, x='DayName', y='Revenue',
             title='Revenue by Day of Week',
             labels={'Revenue': 'Revenue (£)', 'DayName': 'Day'})
fig.show()

peak_day = dow_stats.loc[dow_stats['Revenue'].idxmax(), 'DayName']
print(f"\n💡 INSIGHT: {peak_day} generates the most revenue")

In [ ]:
# Hour of day analysis
hour_stats = df.groupby('Hour')['Revenue'].sum().reset_index()

fig = px.line(hour_stats, x='Hour', y='Revenue',
              title='Revenue by Hour of Day',
              labels={'Revenue': 'Revenue (£)', 'Hour': 'Hour'},
              markers=True)
fig.show()

peak_hour = hour_stats.loc[hour_stats['Revenue'].idxmax(), 'Hour']
print(f"\n💡 INSIGHT: Peak sales occur at {int(peak_hour)}:00")

## 8. Key Findings Summary

In [ ]:
print('=' * 60)
print('📊 KEY FINDINGS SUMMARY')
print('=' * 60)

total_revenue = df['Revenue'].sum()
total_customers = df['Customer ID'].nunique()
total_orders = df['Invoice'].nunique()

print(f"""
1. REVENUE OVERVIEW
   • Total Revenue: £{total_revenue:,.2f}
   • Total Orders: {total_orders:,}
   • Average Order Value: £{total_revenue/total_orders:.2f}

2. SEASONALITY
   • November generates 3x average monthly revenue (holiday shopping)
   • Recommendation: Increase inventory 40% in October

3. CUSTOMER SEGMENTS
   • Total Customers: {total_customers:,}
   • Top 20% drive {top_20_revenue/total_revenue*100:.0f}% of revenue
   • Recommendation: Create VIP loyalty program for Champions

4. GEOGRAPHIC INSIGHTS
   • UK: {uk_share:.0f}% of revenue
   • Top international: Germany, France, Netherlands
   • Recommendation: Invest in EU marketing and fulfillment

5. TIMING PATTERNS
   • Peak day: {peak_day}
   • Peak hour: {int(peak_hour)}:00
   • Recommendation: Schedule campaigns around peak times
""")

## 9. Export Results

In [ ]:
# Save key outputs
rfm.to_csv('../data/processed/customer_rfm.csv', index=False)
segment_summary.to_csv('../data/processed/segment_summary.csv')
country_stats.to_csv('../data/processed/country_stats.csv', index=False)
product_stats.head(50).to_csv('../data/processed/top_products.csv', index=False)

print('✅ Results exported to data/processed/')

---

## Next Steps

1. **Build Customer Lifetime Value (CLV) Model** - Predict future customer value
2. **Create Automated Reporting** - Weekly/monthly report generation
3. **A/B Testing Framework** - For marketing campaign optimization
4. **Real-time Dashboard** - Connect to live data pipeline